In [1]:
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from datasetRegression import EEGDataset # Get the dataset class specific for regression models
from torch.utils.data import random_split
import neptune.new as neptune
from torchinfo import summary

In [2]:
# load in the dataset

#raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'
raw_data_dir = '../data'

trainingNights = 7
testNights = 2

print("Test set")
ds2 = EEGDataset(raw_data_dir,testNights, 250, skips = trainingNights)

print("Training set")
ds1 = EEGDataset(raw_data_dir,trainingNights, 250, skips = 0) #Instantiate a dataset using the directory of data, amount of night to include and amount of samples in a segment



Test set
../data/study_1A_mat_simple/S_01/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_4/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_4/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_4/artefact_annotations.npy
Lables for night 0 loaded
../data/study_1A_mat_sim

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.BatchNorm1d(250),
            nn.Linear(250, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid(), 
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


In [4]:
learning_rate = 1e-3
batch_size = 64


model = NeuralNetwork()
model.to(device)

loss = nn.MSELoss() #L2 loss

#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)# Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
        pred = pred.reshape(-1)
        pred = pred.to(device)
        yFloat = y.type(torch.FloatTensor).to(device)
        
        loss = loss_fn(pred, yFloat)
        


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Neptune logging
        run["training/batch/loss"].log(loss)

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print("Predicted values:")
            print(pred)
            print("Actual values:")
            print(yFloat)



def test_loop(dataloader_test, model, loss_fn, test_set = True):
    size = len(dataloader_test.dataset)
    num_batches = len(dataloader_test)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader_test:
            X = X.to(device)
            y = y.to(device)
            pred = model(X).reshape(-1).to(device) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor).to(device)).item()
            #correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    #correct /= size
    if test_set:
        #print(f"Test set Error: \n Test Set Accuracy: {(100*correct):>0.5f}%, Avg Test Set loss: {test_loss:>8f} \n")
        print(f"Test set Error: \n Avg Test Set loss: {test_loss:>8f} \n")
        # Neptune logging
        run["testing/batch/test_loss"].log(test_loss)
    else:
        #print(f"Training Set Error: \n Training Set Accuracy: {(100*correct):>0.5f}%, Avg Training Set loss: {test_loss:>8f} \n")
        print(f"Training Set Error: \n Avg Training Set loss: {test_loss:>8f} \n")
        # Neptune logging
        run["testing/batch/test_loss"].log(test_loss)


In [6]:
# Split data into train and test data (Commented out: Changed to loading different dataset class instances)
#trainSamples = int(ds1.__len__()*0.7)
#testSamples = int(ds1.__len__() - trainSamples)
#training_data, test_data = random_split(ds1, (trainSamples,testSamples), generator=torch.Generator().manual_seed(42))

#train_dataloader = DataLoader(training_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
#test_dataloader = DataLoader(test_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss


train_dataloader = DataLoader(ds1, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
test_dataloader = DataLoader(ds2, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss



In [7]:
# Initialize neptune
run = neptune.init(
    project="aksel.s.madsen/artefact-detection",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTA4NzcxMy1lYmQ2LTQ3NTctYjRhNC02Mzk1NjdjMWM0NmYifQ==",
)  # Credentials


run['config/dataset/size'] = trainingNights # dict() object
run['config/model'] = type(model).__name__
run['config/modelSummary'] = summary(model, input_size=(batch_size, 250))
run['config/optimizer'] = type(optimizer).__name__
run['config/batch_size'] = batch_size
run['config/test_night'] = testNights


https://app.neptune.ai/aksel.s.madsen/artefact-detection/e/AR-15
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)    
    test_loop(test_dataloader, model, loss)
    #test_loop(train_dataloader, model, loss, test_set = False)

print("Done!")

Epoch 1
-------------------------------
loss: 0.246790  [    0/4951495]
Predicted values:
tensor([0.5063, 0.5084, 0.5064, 0.5082, 0.5069, 0.5082, 0.5028, 0.5087, 0.5055,
        0.5083, 0.5092, 0.5076, 0.5060, 0.5083, 0.5065, 0.5076, 0.5070, 0.5061,
        0.5063, 0.5091, 0.5071, 0.5081, 0.5061, 0.5081, 0.5079, 0.5076, 0.5068,
        0.5084, 0.5076, 0.5083, 0.5067, 0.5083, 0.5042, 0.5085, 0.5057, 0.5090,
        0.5080, 0.5087, 0.5044, 0.5086, 0.5154, 0.5088, 0.4981, 0.5087, 0.5071,
        0.5092, 0.5053, 0.5083, 0.5075, 0.5074, 0.5073, 0.5083, 0.5092, 0.5079,
        0.5026, 0.5085, 0.5076, 0.5081, 0.5104, 0.5084, 0.5220, 0.5083, 0.5140,
        0.5077], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
Actual values:
tensor([1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.00

loss: 0.117759  [512000/4951495]
Predicted values:
tensor([0.4433, 0.2598, 0.9993, 0.2598, 0.4084, 0.2598, 0.8423, 0.2598, 0.9992,
        0.2598, 0.8418, 0.2598, 0.5087, 0.2598, 0.4053, 0.2598, 0.4109, 0.2598,
        0.3151, 0.2598, 0.5064, 0.2598, 0.3422, 0.2598, 0.9209, 0.2598, 0.8621,
        0.2598, 0.7279, 0.2598, 0.5220, 0.2598, 0.4333, 0.2598, 0.9910, 0.2598,
        0.3242, 0.2598, 0.8919, 0.2598, 0.7807, 0.2598, 0.4339, 0.2598, 0.9936,
        0.2598, 0.9702, 0.2598, 0.4305, 0.2598, 0.4941, 0.2598, 1.0000, 0.2598,
        0.5223, 0.2598, 0.2924, 0.2598, 0.9996, 0.2598, 0.4992, 0.2598, 0.3499,
        0.2598], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
Actual values:
tensor([1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 0.6640, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 0.0000, 0.6400, 0.0000, 1.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0

loss: 0.179576  [960000/4951495]
Predicted values:
tensor([0.5281, 0.3714, 0.4211, 0.3419, 0.4345, 0.3405, 0.4313, 0.3384, 0.8491,
        0.3452, 0.9995, 0.3449, 0.8591, 0.3544, 0.8202, 0.3538, 0.5439, 0.3466,
        0.5396, 0.3406, 0.7044, 0.3302, 0.4170, 0.3416, 0.8216, 0.3318, 0.4068,
        0.3389, 0.5618, 0.3373, 1.0000, 0.3592, 0.4504, 0.3584, 0.4800, 1.0000,
        0.8731, 0.3395, 0.5291, 0.3381, 0.9267, 0.3399, 0.7009, 0.3423, 0.3848,
        0.3375, 0.4701, 0.3354, 0.5250, 0.3377, 0.8710, 0.3463, 0.8247, 0.4271,
        0.4871, 1.0000, 0.9799, 0.4132, 0.3854, 0.3726, 0.3970, 0.3535, 0.5006,
        0.3609], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
Actual values:
tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 0.], device='cuda:0')
l

loss: 0.046897  [1408000/4951495]
Predicted values:
tensor([0.7030, 0.1489, 0.3972, 0.1489, 0.9937, 0.1489, 0.6683, 0.1489, 1.0000,
        0.1489, 0.9454, 0.1489, 0.9983, 0.1489, 0.9998, 0.1489, 1.0000, 0.1489,
        0.5555, 0.1489, 0.8879, 0.1489, 0.7286, 0.1489, 0.3577, 0.1489, 1.0000,
        0.1489, 0.9999, 0.1489, 0.4877, 0.1489, 0.6621, 0.1489, 0.6985, 0.1489,
        0.7866, 0.1489, 0.5435, 0.1489, 0.3910, 0.1489, 1.0000, 0.1489, 0.9926,
        0.1489, 0.9040, 0.1489, 0.9998, 0.1489, 0.9948, 0.1489, 1.0000, 0.1489,
        1.0000, 0.1489, 0.8621, 0.1489, 0.9676, 0.1489, 0.8604, 0.1489, 1.0000,
        0.1489], device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
Actual values:
tensor([1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.9320, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.9560, 0.0000, 1.

In [ ]:
run.stop() # Stop the neptune logging run